## PS1: Optimize the feature set of a rack-mounted M2 MacPro Server
The [new MacPro with the M2 Ultra chip has been released](https://www.apple.com/shop/buy-mac/mac-pro/rack#). The MacPro M2 has several configuration options broadly organized into five categories: `{CPU, Memory, Storage, Accessories, Software}` with multiple options per category:
* The `CPU` category has `2` options. Only one option can be selected from the `CPU` category
* The `Memory` category has `3` options. Only one option can be selected from the `Memory` category.
* The `Storage` category has `4` options. Only one option can be selected from the `Storage` category.
* The `Accessories` category has `3` options. Only one option can be selected from the `Accessories` category
* The `Software` category has `2` options. Neither or both options can be selected from the `Software` category

### Problem statement
Using a `linear` utility model and budget and feature constraints, compute the optimal set of features for a rack-mounted MacPro M2. This problem will be similar to the examples discussed in the lecture and discussion, except for one crucial element: the decision variables (our choices) will be `binary`, i.e., $x_{i}\in{0,1}$ where a `0` indicates that we do NOT choose feature $i$. In contrast, a value of `1` indicates that we choose option $i$. 

Formally, an agent has a set of $n$ configuration options $X = \left\{x_{i}\right\}_{i=1}^{n}$, a `Linear` utility function, and a total of $I$ units of resource to allocate, e.g., money, and potentially other constraints. An optimal agent maximizes its utility subject to its resource and other constraints:

\begin{eqnarray}
\text{maximize}~\mathcal{O} &=& \sum_{i\in{1,\dotsc,n}}\alpha_{i}x_{i} \\
\text{subject to}~\sum_{i\in{1,\dotsc,n}}c_{i}x_{i} & = & I\\
\text{and}~\mathbf{C}\mathbf{x} & \leq & \mathbf{b} \\
\text{and}~x_{i}&\in&{0,1}\qquad{i=1,2,\dots,n}
\end{eqnarray}

The quantity $c_{i}\geq{0}~\forall{i}$ denotes the cost of option $i$, $\alpha_{i}$ denotes user-specified coefficient in the `Linear` utility function, $x_{i}\in{0,1}$ represents the choice of option $i$, and $\mathbf{C}\mathbf{x} \leq \mathbf{b}$ represents additional constraints governing the decision.

#### List of Tasks
* __Task 1__: Specify $\alpha$- and $c$-vectors for the problem
* __Task 2__: Specify the additional constraint matrix $\mathbf{C}$
* __Task 3__: Specify the problem object and solve the problem
* __Task 4__: Try at least two different weighting schemes and different budget values, and explore how these design choices influence the optimal configuration (one of these __can__ be the `default` values specified below).

### Setup
The computations in this problem set rely on [VLDecisionsPackage.jl](https://github.com/varnerlab/VLDecisionsPackage.jl.git) and several external `Julia` packages. To load the required packages and any custom codes the teaching team has developed to work with these packages, we [include](https://docs.julialang.org/en/v1/manual/code-loading/) the `Include.jl` file):

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-5760-PS1-OptimalMacProRackDesign-Fall-2023`
Precompiling project...
  ✓ Cairo_jll
  ✓ HarfBuzz_jll
  ✓ libass_jll
  ✓ Optim
  ✓ HTTP
  ✓ FFMPEG_jll
  ✓ FFMPEG
  ✓ GR_jll
  ✓ Distributions
  ✓ GR
  ✓ Plots
  ✓ Plots → UnitfulExt
  ✓ MathOptInterface
  ✓ MadNLP
  ✓ GLPK
  ✓ JuMP
  ✓ VLDecisionsPackage
  17 dependencies successfully precompiled in 145 seconds. 196 already precompiled.
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Installed Scratch ─ v1.2.1
    Updating `~/Desktop/julia_work/CHEME-5760-PS1-OptimalMacProRackDesign-Fall-2023/Project.toml`
  [10f378ab] ~ VLDecisionsPackage v0.1.0 `https://github.com/varnerlab/VLDecisionsPackage.jl.git#main` ⇒ v0.1.0 `https://github.com/varnerlab/VLDecisionsPackage.jl.git#main`
    Updating `~/Desktop/julia_work/CHEME-5760-PS1-OptimalMacProRackDesign-Fall-2023/Manifest.toml`
  [14f7f29c] ↑ AMD v0.5

### Some additional setup stuff
We’ll use `DataFrames` in the code below to display our choice data. 
* The `DataFrame` type is exported by the [DataFrames.jl](https://dataframes.juliadata.org/stable/) package, which provides a set of tools for working with tabular data in [Julia](https://julialang.org). Its design and functionality are similar to those of [Pandas (in Python)](https://pandas.pydata.org) and [data.frame, data.table, and dplyr (in R)](https://dplyr.tidyverse.org), making it an excellent general-purpose data science tool.
* We add `DataFrames` to the notebook by issuing the `using Pkg` command followed by the `Pkg.add("DataFrames")` command. [Pkg.jl](https://pkgdocs.julialang.org/v1/getting-started/) is the `built-in` Julia package manager used to `add/delete` external package dependencies. These two commands load `Pkg`, and use it to add `DataFrames` to the notebook. 
* Finally, the `using DataFrames` command loads the `DataFrames` package so we can use it.

In [2]:
# extra stuff that I'm adding -
using Pkg;
Pkg.add("DataFrames")
using DataFrames

   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-PS1-OptimalMacProRackDesign-Fall-2023/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-PS1-OptimalMacProRackDesign-Fall-2023/Manifest.toml`


#### Specify constants and other static stuff

In [3]:
number_of_choices = 14;
bounds_array = Array{Float64,2}(undef, number_of_choices,2)
for i ∈ 1:number_of_choices
    bounds_array[i,1] = 0.0
    bounds_array[i,2] = 1.0;
end

We add the `label_dictionary` dictionary which holds the labels for the options in our choice set:

In [4]:
label_dictionary = Dict{Int64,String}()
label_dictionary[1] = "CPU 1"
label_dictionary[2] = "CPU 2"
label_dictionary[3] = "Memory 1"
label_dictionary[4] = "Memory 2"
label_dictionary[5] = "Memory 3"
label_dictionary[6] = "Storage 1"
label_dictionary[7] = "Storage 2"
label_dictionary[8] = "Storage 3"
label_dictionary[9] = "Storage 4"
label_dictionary[10] = "Accessory 1"
label_dictionary[11] = "Accessory 2"
label_dictionary[12] = "Accessory 3"
label_dictionary[13] = "Software 1"
label_dictionary[14] = "Software 2";

## Task 1: Specify the configuration array
The `configuration_array` is a `14` $\times$ `2` array holding perception and cost information about the problem. Each row of the `configuration_array` contains data for a particular MacPro configuration option. The first column contains the coefficients of the `Linear` utility function, i.e., the elements of the $\alpha$-vector, while the unit price of the features, i.e., elements of the $c$-vector, are in the second column.  

#### Feature prices
The prices of each configuration option have been estimated from the [Apple MacPro server website](https://www.apple.com/shop/buy-mac/mac-pro/rack#).

#### Linear utility weighting scheme
In `PS1`, we'll use a category-based weighting scheme. In each of the five categories, allocate `1` unit of weight:

* In each category, the coefficients in the `Linear` utility function for options in that category must sum to one.

For example, if you have equal feelings about three options in a category, then `0.33, 0.33, 0.33` would be a typical scheme. On the other hand, if you are excited about feature `1` over the other two in the category, then `0.8,0.1,0.1` could be an appropriate weight scheme.

In [5]:
configuration_array = [

    # 1 CPU options
    0.5 2640.0    ; # 1 CPU 1
    0.5 3649.0    ; # 2 CPU 2

    # 2 Memory options
    0.333 3840.0  ; # 3 Memory 1
    0.333 4640.0  ; # 4 Memory 2
    0.333 3600.0  ; # 5 Memory 3

    # 3 Storage options -
    0.25 1440.0   ; # 6 Storage 1
    0.25 1840.0   ; # 7 Storage 2
    0.25 2440.0   ; # 8 Storage 3
    0.25 3640.0   ; # 9 Storage 4

    # 4 Accessory options
    0.333 79.0    ; # 10 Accessory 1
    0.333 129.0   ; # 11 Accessory 2
    0.333 149.0   ; # 12 Accessory 3
    
    # Software options
    0.5 299.0     ; # 13 Software 1
    0.5 149.0     ; # 14 Software 2
];

### Task 2: Specify the feature constraint matrix `C`
In each category, only a finite number of options can be selected simultaneously, typically only a single option, with the exception being the `Software` category, which is unconstrained (can have from zero up to two items selected). Because the decision variables are binary, we can implement this requirement with an additional set of constraints of the form:

$$
\begin{equation}
\mathbf{C}\mathbf{x} = \mathbf{1}
\end{equation}
$$

where $\mathbf{C}$ denotes the constraint matrix, $\mathbf{x}$ denotes the choice vector and $\mathbf{1}$ denotes a vector of `1`'s. 
Specify the $\mathbf{C}$ matrix for this problem:

In [6]:
C = zeros(4,14); # understanding qestion: why is this a 4 x 14 array?

In [7]:
C[1,1:2] .= 1;    # constraints on CPU options
C[2,3:5] .= 1;    # constraints on memory options
C[3,6:9] .= 1;    # constraints on storage options
C[4,10:12] .= 1;  # constraints on accessory options

__Note__: Julia's Array syntax is similar to Matlab/Octave, except with square brackets. See [the Array documentation](https://docs.julialang.org/en/v1/base/arrays/) or various other [Julia tutorials on the web](https://www.tutorialspoint.com/julia/julia_arrays.htm) about working with the Array data structure.

### Task 3: Specify the problem object, and solve for the optimal configuration
Finally, build a `MySimpleBinaryVariableLinearChoiceProblem` model using the `build(...)` method, set this instance to the `problem` variable, and then pass the `problem` object to the `solve(...)` method. The `solve(...)` method will solve the `ILP` problem using the [GLPK.jl](https://github.com/jump-dev/GLPK.jl) interface to the [GLPK linear programming solver](https://www.gnu.org/software/glpk/).

In [8]:
α = configuration_array[:,1];
c = configuration_array[:,2];
I = 10000; # default budget is 10K USD

In [9]:
problem = build(MySimpleBinaryVariableLinearChoiceProblem, (
    
    α = α,
    c = c,
    I = I,
    initial = zeros(14),
    bounds = bounds_array,

    # extra constraints -
    C = C
));
solution = solve(problem)

Dict{String, Any} with 3 entries:
  "argmax"          => [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, …
  "budget"          => 8447.0
  "objective_value" => 2.416

#### Check: Are the choice constraints enforced?
We can only select a fixed number of items from each category. Does your solution reflect this restriction? If the choice constraints are enforced, then the product of the solution and the constraint matrix should be the `4`$\times$`1` vector of `1's`:

In [10]:
x = solution["argmax"]

14-element Vector{Float64}:
 1.0
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 1.0
 1.0

In [11]:
C*x

4-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0

### Task 4: How does changing the $\alpha$-vector (or the budget $I$) influence the configuration choice?
Let's explore two cases which mimics how I purchase Apple products, namely:
* `Case 1`: I maximize all the options (and hit my budget constraint)
* `Case 2`: I maximize all the options (hit my budget constraint), but spend more than I wanted to (increase the budget constraint)

#### Case 1: Value the highest performance selections the most (change the $\alpha$ values) with the original budget
First, let's create a new configuration array named `configuration_array_case_1` in which we value the most expensive element of each category:

In [12]:
configuration_array_case_1 = [

    # 1 CPU options
    0.05 2640.0    ; # 1 CPU 1
    0.95 3649.0    ; # 2 CPU 2

    # 2 Memory options
    0.1 3840.0  ; # 3 Memory 1
    0.1 4640.0  ; # 4 Memory 2
    0.8 3600.0  ; # 5 Memory 3

    # 3 Storage options -
    0.1 1440.0   ; # 6 Storage 1
    0.1 1840.0   ; # 7 Storage 2
    0.1 2440.0   ; # 8 Storage 3
    0.7 3640.0   ; # 9 Storage 4

    # 4 Accessory options
    0.1 79.0    ; # 10 Accessory 1
    0.1 129.0   ; # 11 Accessory 2
    0.8 149.0   ; # 12 Accessory 3
    
    # Software options
    0.1 299.0     ; # 13 Software 1
    0.9 149.0     ; # 14 Software 2
];

Next, create a new problem object using the updated configuration array using the `build(...)` method and assign it to the `problem_case_1` variable. Then, solve the problem using the `solve(...)` method. Assign the solution to the `solution_case_1` variable:

In [13]:
α₁ = configuration_array_case_1[:,1];
c₁ = configuration_array_case_1[:,2];
problem_case_1 = build(MySimpleBinaryVariableLinearChoiceProblem, (
    
    α = α₁,
    c = c₁,
    I = I,
    initial = zeros(14),
    bounds = bounds_array,

    # extra constraints -
    C = C
));
solution_case_1 = solve(problem_case_1)

Dict{String, Any} with 3 entries:
  "argmax"          => [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, …
  "budget"          => 9686.0
  "objective_value" => 3.65

In [14]:
x₁ = solution_case_1["argmax"]

14-element Vector{Float64}:
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0

In [15]:
C*x₁

4-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0

#### Visualize: Build a table with the selected options for `case 1`
Let's build a table that holds the selected options for `case 1` and their prices using a `DataFrame` instance:

In [16]:
table_case_1 = DataFrame()
for i ∈ eachindex(x₁)
    
    choice_value = x₁[i]
    if (choice_value == 1)
        
        # row data -
        row_data = (
            option = label_dictionary[i],
            price = c₁[i]
        )
        push!(table_case_1, row_data)
    end
end
table_case_1

Row,option,price
,String,Float64
1,CPU 2,3649.0
2,Memory 3,3600.0
3,Storage 2,1840.0
4,Accessory 3,149.0
5,Software 1,299.0
6,Software 2,149.0


In [17]:
total_cost = sum(table_case_1[:,:price])

9686.0

#### Case 2: Value the highest performance selections the most, and increase the budget to `I = 15000 USD`
Create a new problem object using the updated budget value `I = 15000` and configuration array from `case 1` using the `build(...)` method and assign it to the `problem_case_2` variable. Then, solve the problem using the `solve(...)` method. Assign the solution to the `solution_case_2` variable:

In [18]:
α₂ = configuration_array_case_1[:,1];
c₂ = configuration_array_case_1[:,2];
problem_case_2 = build(MySimpleBinaryVariableLinearChoiceProblem, (
    
    α = α₂,
    c = c₂,
    I = 25000,
    initial = zeros(14),
    bounds = bounds_array,

    # extra constraints -
    C = C
));
solution_case_2 = solve(problem_case_2)

Dict{String, Any} with 3 entries:
  "argmax"          => [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, …
  "budget"          => 11486.0
  "objective_value" => 4.25

In [19]:
x₂ = solution_case_2["argmax"]

14-element Vector{Float64}:
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0

#### Visualize: Build a table with the selected options for `case 2`
Let's build a table that holds the selected options for `case 2` and their prices using a `DataFrame` instance:

In [20]:
table_case_2 = DataFrame()
for i ∈ eachindex(x₂)
    
    choice_value = x₂[i]
    if (choice_value == 1)
        
        # row data -
        row_data = (
            option = label_dictionary[i],
            price = c₂[i]
        )
        push!(table_case_2, row_data)
    end
end
table_case_2

Row,option,price
,String,Float64
1,CPU 2,3649.0
2,Memory 3,3600.0
3,Storage 4,3640.0
4,Accessory 3,149.0
5,Software 1,299.0
6,Software 2,149.0


In [21]:
total_cost = sum(table_case_2[:,:price])

11486.0